In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import math
import numpy as np


Bad key text.latex.preview in file /Users/thomasat/opt/miniconda3/envs/nhanes/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file /Users/thomasat/opt/miniconda3/envs/nhanes/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.jpeg_quality in file /Users/thomasat/opt/miniconda3/envs/nhanes/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 418 ('savefig.jpeg_

In [2]:
plt.rcParams['font.family'] = 'Helvetica'
units = 'weeks'
model = 'bin'
slope = True

if units == 'weeks':
    if model == 'bin' and slope:
        results_dir = '../results/bin_5months_slopeTrue_weeks_2024-06-30_09_28_11.630821/'
    elif model == 'bin' and (not slope):
        results_dir = '../results/bin_5months_slopeFalse_weeks_2024-06-30_09_26_57.885004/'
    else:
        results_dir = '../results/assoc_contemp_lagged_2024_weeks_2024-06-26_15_47_00.166579/'
    #results_dir = '../results/assoc_contemp_lagged_weeks_2024-05-11_18_47_40.615096/'
else:
    if model == 'bin' and slope:
        results_dir = '../results/bin_5months_slopeTrue_months_2024-06-30_09_10_38.210285/'
    elif model == 'bin' and (not slope):
        results_dir = '../results/bin_5months_slopeFalse_months_2024-06-30_09_08_51.464113/'
    else:    
        results_dir = '../results/assoc_contemp_lagged_2024_months_2024-06-26_15_35_52.478345/'
    #results_dir = '../results/assoc_contemp_lagged_months_2024-05-11_18_44_16.131608/'

#results_dir = '../results/bin_no_differencing_5months_months_2024-05-23_10_09_14.130978/'

bonf_denom = 6
if units == 'weeks':
    bonf_denom = 7

In [3]:
"""
Assumes SD of outcome is 1, and true is 0, and contrast of interest in exposure is 1.
"""
def compute_eval_pe(est):
    # Risk ratio
    rr = np.exp(0.91*np.abs(est))
    return rr + np.sqrt(rr*(rr - 1))
    
def compute_eval_ci(est, se):
    if est - 1.96*se <= 0:
        return 1
    
    ci_rr_lo = np.exp(0.91*est - 1.78*se)
    ci_rr_hi = np.exp(0.91*est + 1.78*se)
    
    #print(ci_lo, ci_hi)
    
    if ci_rr_lo <= 0:
        return 1
    
    e_val = min(ci_rr_lo + np.sqrt(ci_rr_lo*(ci_rr_lo - 1)), ci_rr_hi + np.sqrt(ci_rr_hi*(ci_rr_hi - 1)))
    
    return e_val

In [4]:
#assoc_df = pd.read_csv(results_dir + 'assoc_df_{u}_all_primary.csv'.format(u=units))
#lagged_df = pd.read_csv(results_dir + 'lagged_df_{u}_all_primary.csv'.format(u=units))
#contemp_df = pd.read_csv(results_dir + 'contemp_df_{u}_all_primary.csv'.format(u=units))
#bin_df = pd.read_csv(results_dir + 'bin_df_{u}_all_primary.csv'.format(u=units))

df = pd.read_csv(results_dir + model + '_df_{u}_all_primary.csv'.format(u=units))

df = df.loc[df['Outcome'] != 'NeuhoferLusk']

In [5]:
df

,Outcome,FOK_se,Cowspiracy_se,Okja/WTH_se,TGC_se,YAWYE_se,All_se,FOK_pe,Cowspiracy_pe,Okja/WTH_pe,TGC_pe,YAWYE_pe,All_pe,FOK_pval,Cowspiracy_pval,Okja/WTH_pval,TGC_pval,YAWYE_pval,All_pval
0,Searches: `Plant based',0.098035,0.043491,0.060716,0.054028,0.036170,NaN,0.567157,0.067588,0.132625,0.114280,-0.134581,NaN,1.506495e-08,0.120685,0.029267,0.034714,2.092320e-04,NaN
1,Searches: `Vegan',0.061053,0.044642,0.050009,0.043017,0.036889,NaN,0.066380,0.164903,0.167144,0.055975,-0.106846,NaN,2.776074e-01,0.000241,0.000875,0.193543,3.852946e-03,NaN
2,Searches: `Vegetarian',0.120438,0.079533,0.066813,0.057006,0.062841,NaN,-0.021465,0.174627,0.136620,0.022956,-0.183411,NaN,8.586414e-01,0.028491,0.041247,0.687278,3.591058e-03,NaN
3,StewartMilk,NaN,0.081834,0.076825,NaN,NaN,NaN,NaN,0.098965,-0.001239,NaN,NaN,NaN,NaN,0.228600,0.987150,NaN,NaN,NaN
4,StewartPBMilk,NaN,0.098858,0.079299,NaN,NaN,NaN,NaN,-0.077601,-0.004939,NaN,NaN,NaN,NaN,0.433814,0.950389,NaN,NaN,NaN
5,Zhao,NaN,NaN,0.049057,0.044463,NaN,NaN,NaN,NaN,0.003956,-0.016527,NaN,NaN,NaN,NaN,0.936403,0.710662,NaN,NaN
6,"Searches: `Vegan', Informative",0.087349,0.056574,0.061572,0.055185,0.044466,NaN,0.050235,0.215562,0.174126,0.061334,-0.090975,NaN,5.655550e-01,0.000153,0.004818,0.266708,4.101337e-02,NaN
7,"Searches: `Vegetarian', Informative",0.101743,0.079553,0.077160,0.073424,0.071254,NaN,-0.035564,0.266160,0.200588,0.030307,-0.154100,NaN,7.268705e-01,0.000871,0.009529,0.679881,3.079213e-02,NaN
8,"Searches: `Plant based', Informative",0.097041,0.041460,0.058802,0.053943,0.036211,NaN,0.475138,0.055749,0.139343,0.112299,-0.116541,NaN,1.442737e-06,0.179238,0.018072,0.037669,1.329015e-03,NaN
9,"Searches: `Vegan', Behavior",0.038186,0.036582,0.036453,0.035487,0.031410,NaN,0.072805,0.023711,0.178256,0.050107,-0.123232,NaN,5.732429e-02,0.517129,0.000001,0.158327,9.306816e-05,NaN


In [6]:
df['Outcome'] = df['Outcome'].replace({'Zhao': 'Plant-based meat sales',
                                      'StewartPBMilk': 'Plant-based milk sales',
                                      'StewartMilk': 'Milk sales'})
df

,Outcome,FOK_se,Cowspiracy_se,Okja/WTH_se,TGC_se,YAWYE_se,All_se,FOK_pe,Cowspiracy_pe,Okja/WTH_pe,TGC_pe,YAWYE_pe,All_pe,FOK_pval,Cowspiracy_pval,Okja/WTH_pval,TGC_pval,YAWYE_pval,All_pval
0,Searches: `Plant based',0.098035,0.043491,0.060716,0.054028,0.036170,NaN,0.567157,0.067588,0.132625,0.114280,-0.134581,NaN,1.506495e-08,0.120685,0.029267,0.034714,2.092320e-04,NaN
1,Searches: `Vegan',0.061053,0.044642,0.050009,0.043017,0.036889,NaN,0.066380,0.164903,0.167144,0.055975,-0.106846,NaN,2.776074e-01,0.000241,0.000875,0.193543,3.852946e-03,NaN
2,Searches: `Vegetarian',0.120438,0.079533,0.066813,0.057006,0.062841,NaN,-0.021465,0.174627,0.136620,0.022956,-0.183411,NaN,8.586414e-01,0.028491,0.041247,0.687278,3.591058e-03,NaN
3,Milk sales,NaN,0.081834,0.076825,NaN,NaN,NaN,NaN,0.098965,-0.001239,NaN,NaN,NaN,NaN,0.228600,0.987150,NaN,NaN,NaN
4,Plant-based milk sales,NaN,0.098858,0.079299,NaN,NaN,NaN,NaN,-0.077601,-0.004939,NaN,NaN,NaN,NaN,0.433814,0.950389,NaN,NaN,NaN
5,Plant-based meat sales,NaN,NaN,0.049057,0.044463,NaN,NaN,NaN,NaN,0.003956,-0.016527,NaN,NaN,NaN,NaN,0.936403,0.710662,NaN,NaN
6,"Searches: `Vegan', Informative",0.087349,0.056574,0.061572,0.055185,0.044466,NaN,0.050235,0.215562,0.174126,0.061334,-0.090975,NaN,5.655550e-01,0.000153,0.004818,0.266708,4.101337e-02,NaN
7,"Searches: `Vegetarian', Informative",0.101743,0.079553,0.077160,0.073424,0.071254,NaN,-0.035564,0.266160,0.200588,0.030307,-0.154100,NaN,7.268705e-01,0.000871,0.009529,0.679881,3.079213e-02,NaN
8,"Searches: `Plant based', Informative",0.097041,0.041460,0.058802,0.053943,0.036211,NaN,0.475138,0.055749,0.139343,0.112299,-0.116541,NaN,1.442737e-06,0.179238,0.018072,0.037669,1.329015e-03,NaN
9,"Searches: `Vegan', Behavior",0.038186,0.036582,0.036453,0.035487,0.031410,NaN,0.072805,0.023711,0.178256,0.050107,-0.123232,NaN,5.732429e-02,0.517129,0.000001,0.158327,9.306816e-05,NaN


In [7]:
df.filter(regex='FOK|Outcome')

,Outcome,FOK_se,FOK_pe,FOK_pval
0,Searches: `Plant based',0.098035,0.567157,1.506495e-08
1,Searches: `Vegan',0.061053,0.066380,2.776074e-01
2,Searches: `Vegetarian',0.120438,-0.021465,8.586414e-01
3,Milk sales,NaN,NaN,NaN
4,Plant-based milk sales,NaN,NaN,NaN
5,Plant-based meat sales,NaN,NaN,NaN
6,"Searches: `Vegan', Informative",0.087349,0.050235,5.655550e-01
7,"Searches: `Vegetarian', Informative",0.101743,-0.035564,7.268705e-01
8,"Searches: `Plant based', Informative",0.097041,0.475138,1.442737e-06
9,"Searches: `Vegan', Behavior",0.038186,0.072805,5.732429e-02


In [8]:
def produce_evalues_df(df):
    if units == 'months' or model == 'bin':
        intvns = ['FOK', 'Cowspiracy', 'Okja/WTH', 'TGC', 'YAWYE']
    else:
        intvns = ['FOK', 'Cowspiracy', 'WTH', 'Okja', 'TGC', 'YAWYE']        
    dct = {}
    for intvn in intvns:
        dct[intvn] = {}
        for outcome in df['Outcome']:
            pe = df.loc[df['Outcome'] == outcome][intvn + '_pe'].item()
            se = df.loc[df['Outcome'] == outcome][intvn + '_se'].item()
            if math.isnan(pe) and math.isnan(se):
                dct[intvn][outcome] = ''                
            else:
                eval_pe = compute_eval_pe(pe)
                eval_ci = compute_eval_ci(pe, se)
                dct[intvn][outcome] = '({pe:.2f}, {ci:.2f})'.format(pe=eval_pe,ci=eval_ci)

            
    table_df = pd.DataFrame(dct)
                    
    return table_df

In [9]:
"""
Outcome | FOK | Cowspiracy | WTH | Okja | TGC
"""
def produce_df(df, bonf_denom, alpha=0.05):
    if units == 'months' or model == 'bin':
        intvns = ['FOK', 'Cowspiracy', 'Okja/WTH', 'TGC' ,'YAWYE']
    else:
        intvns = ['FOK', 'Cowspiracy', 'WTH', 'Okja', 'TGC', 'YAWYE']        
    list_to_bold = []
    dct = {}
    for intvn in intvns:
        dct[intvn] = {}
        for outcome in df['Outcome']:
            pe = df.loc[df['Outcome'] == outcome][intvn + '_pe'].item()
            pval = df.loc[df['Outcome'] == outcome][intvn + '_pval'].item()
            if math.isnan(pe) and math.isnan(pval):
                dct[intvn][outcome] = ''                
            elif pval < 0.1:
                dct[intvn][outcome] = '{pe:.2f} ({p:.2e})'.format(pe=pe,p=pval)
            else:
                dct[intvn][outcome] = '{pe:.2f} ({p:.2f})'.format(pe=pe,p=pval)
                
                
            if pval < alpha/bonf_denom:
                list_to_bold.append('{pe:.2f}'.format(pe=pe))
                
            
    table_df = pd.DataFrame(dct)
                    
    return table_df, list_to_bold
        

In [10]:
tab_str = produce_evalues_df(df)
tab_str = tab_str.to_latex()
tab_str = tab_str.replace('{lllllll}', '{|l|l|l|l|l|l|l|}')
tab_str = tab_str.replace('{llllll}', '{|l|l|l|l|l|l|}')
tab_str = tab_str.replace('{lllll}', '{|l|l|l|l|l|}')
tab_str = tab_str.replace('{}', 'Outcome')
tab_str = tab_str.replace('toprule', 'hline')
tab_str = tab_str.replace('midrule', 'hline')
tab_str = tab_str.replace('bottomrule', 'hline')
tab_str = tab_str.replace('Okja/WTH', 'WTH/Okja')
for outcome in df['Outcome']:
    tab_str = tab_str.replace(' \\\\\n' + outcome, ' \\\\ \\hline\n' + outcome)
print(tab_str)

\begin{tabular}{|l|l|l|l|l|l|}
\hline
Outcome &           FOK &    Cowspiracy &      WTH/Okja &           TGC &         YAWYE \\
\hline
Searches: `Plant based'              &  (2.74, 2.16) &  (1.32, 1.00) &  (1.51, 1.13) &  (1.46, 1.10) &  (1.51, 1.00) \\ \hline
Searches: `Vegan'                    &  (1.32, 1.00) &  (1.60, 1.35) &  (1.60, 1.33) &  (1.29, 1.00) &  (1.44, 1.00) \\ \hline
Searches: `Vegetarian'               &  (1.16, 1.00) &  (1.62, 1.15) &  (1.52, 1.08) &  (1.17, 1.00) &  (1.64, 1.00) \\ \hline
Milk sales                           &               &  (1.42, 1.00) &  (1.03, 1.00) &               &               \\ \hline
Plant-based milk sales               &               &  (1.35, 1.00) &  (1.07, 1.00) &               &               \\ \hline
Plant-based meat sales               &               &               &  (1.06, 1.00) &  (1.14, 1.00) &               \\ \hline
Searches: `Vegan', Informative       &  (1.27, 1.00) &  (1.73, 1.43) &  (1.62, 1.28) &  (1.30, 1.00) &

In [11]:
tab_str, list_to_bold = produce_df(df, bonf_denom)
tab_str

,FOK,Cowspiracy,Okja/WTH,TGC,YAWYE
Searches: `Plant based',0.57 (1.51e-08),0.07 (0.12),0.13 (2.93e-02),0.11 (3.47e-02),-0.13 (2.09e-04)
Searches: `Vegan',0.07 (0.28),0.16 (2.41e-04),0.17 (8.75e-04),0.06 (0.19),-0.11 (3.85e-03)
Searches: `Vegetarian',-0.02 (0.86),0.17 (2.85e-02),0.14 (4.12e-02),0.02 (0.69),-0.18 (3.59e-03)
Milk sales,,0.10 (0.23),-0.00 (0.99),,
Plant-based milk sales,,-0.08 (0.43),-0.00 (0.95),,
Plant-based meat sales,,,0.00 (0.94),-0.02 (0.71),
"Searches: `Vegan', Informative",0.05 (0.57),0.22 (1.53e-04),0.17 (4.82e-03),0.06 (0.27),-0.09 (4.10e-02)
"Searches: `Vegetarian', Informative",-0.04 (0.73),0.27 (8.71e-04),0.20 (9.53e-03),0.03 (0.68),-0.15 (3.08e-02)
"Searches: `Plant based', Informative",0.48 (1.44e-06),0.06 (0.18),0.14 (1.81e-02),0.11 (3.77e-02),-0.12 (1.33e-03)
"Searches: `Vegan', Behavior",0.07 (5.73e-02),0.02 (0.52),0.18 (1.25e-06),0.05 (0.16),-0.12 (9.31e-05)


In [203]:
tab_str, list_to_bold = produce_df(df, bonf_denom)
print(list_to_bold)
tab_str = tab_str.to_latex()
tab_str = tab_str.replace('{lllllll}', '{|l|l|l|l|l|l|l|}')
tab_str = tab_str.replace('{llllll}', '{|l|l|l|l|l|l|}')
tab_str = tab_str.replace('{lllll}', '{|l|l|l|l|l|}')
tab_str = tab_str.replace('{}', 'Outcome')
tab_str = tab_str.replace('toprule', 'hline')
tab_str = tab_str.replace('midrule', 'hline')
tab_str = tab_str.replace('bottomrule', 'hline')
tab_str = tab_str.replace('Okja/WTH', 'WTH/Okja')
for outcome in df['Outcome']:
    tab_str = tab_str.replace(' \\\\\n' + outcome, ' \\\\ \\hline\n' + outcome)
for num in list_to_bold:
    tab_str = tab_str.replace(num + ' (', '\\textbf{' + num + '} (')

print(tab_str)

['0.57', '0.48', '1.13', '0.16', '0.22', '0.27', '0.17', '0.17', '0.18', '0.20', '-0.13', '-0.11', '-0.18', '-0.12', '-0.12', '-0.29']
\begin{tabular}{|l|l|l|l|l|l|}
\hline
Outcome &              FOK &       Cowspiracy &         WTH/Okja &              TGC &             YAWYE \\
\hline
Searches: `Plant based'              &  \textbf{0.57} (1.51e-08) &      0.07 (0.12) &  0.13 (2.93e-02) &  0.11 (3.47e-02) &  \textbf{-0.13} (2.09e-04) \\ \hline
Searches: `Vegan'                    &      0.07 (0.28) &  \textbf{0.16} (2.41e-04) &  \textbf{0.17} (8.75e-04) &      0.06 (0.19) &  \textbf{-0.11} (3.85e-03) \\ \hline
Searches: `Vegetarian'               &     -0.02 (0.86) &  \textbf{0.17} (2.85e-02) &  0.14 (4.12e-02) &      0.02 (0.69) &  -\textbf{0.18} (3.59e-03) \\ \hline
Milk sales                           &                  &      0.10 (0.23) &     -0.00 (0.99) &                  &                   \\ \hline
Plant-based milk sales               &                  &     -0.08 (0.43) &  